### Chains with multiple inputs

In [6]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [7]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

APIConnectionError: Connection error.

In [ ]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "hindi"})

{'input': 'a parrot',
 'language': 'hindi',
 'text': 'एक आदमी के पास एक तोता था। एक दिन वो अपने दोस्त से बोला, "यार, मेरा तोता बहुत बुद्धिमान है!"\n\nदोस्त ने पूछा, "क्यों? वो क्या करता है?"\n\nआदमी बोला, "जब मैं उसे \'तोता\' कहता हूँ, वो तुरंत जवाब देता है \'हाँ, मैं हूँ!\'"\n\nदोस्त हंसते हुए बोला, "पर वो तो बता रहा है कि वो तोता है, तुम्हें देखकर क्या बोलता है?"\n\nआदमी ने कहा, "वो तब चुप रहता है। उसे पता है कितनी बार मैंने \'निहारे\' हैं!"'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [ ]:
from langchain.chains.sequential import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} from {restaurantName} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

#=============================================================================================================

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

#==============================================================================================================

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

#================================================================================================================

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="""Translate the summary in 3 languages 
    
    1. Hindi
    2. French
    3. Spanish 
    
    \n\n 
    
    {summary}
    """
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="hindi_translation"
)

#==================================================================================================================

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name","restaurantName","experience"],
    output_variables=["review", "comment", "summary", "hindi_translation"],
)

overall_chain.invoke({"dish_name": "Chicken Biryani", "restaurantName":"Akbar Biryani", "experience": "It was awful!"})

{'dish_name': 'Chicken Biryani',
 'restaurantName': 'Akbar Biryani',
 'experience': 'It was awful!',
 'review': '**Review of Akbar Biryani: A Disappointing Experience**\n\nI recently ordered Chicken Biryani from Akbar Biryani, and I have to say, my experience was nothing short of awful. \n\nFirst off, the delivery was delayed, which set the tone for the entire meal. When the food finally arrived, it was lukewarm at best, which is never a good sign for a dish that should be served hot and fresh.\n\nNow, onto the biryani itself. The chicken was overcooked and rubbery, lacking any of the tenderness I expected. The rice was underseasoned and lacked the aromatic spices that are the hallmark of a good biryani. It felt more like a bland rice dish than the flavorful feast that biryani should be. Additionally, there seemed to be an alarming absence of essential ingredients like saffron or fried onions, which typically elevate this dish.\n\nThe portion size was another letdown. For the price I p